<a href="https://colab.research.google.com/github/AKUG47/BPE-pytorch/blob/master/softmax_linreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>